#### Файл сервера xmlrpc_stats_server_p3.ipynb на Python 3 

In [ ]:

from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler
import os
import pandas as pd

import datetime as dt

LIMIT_LINE = 100


class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = SimpleXMLRPCServer(("localhost", 8018), 
                            requestHandler=RequestHandler)


# Добавление строки в лог
def add_log(sname):
    global LIMIT_LINE
    with open("log.csv", "r") as file1:
        line_list = file1.readlines()
    if len(line_list) >= LIMIT_LINE:
        date_save = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
        with open(f'logs/{date_save}.csv', "w") as file2:
            file2.write("".join(line_list))
        with open("log.csv", "w") as file3:
            pass
    f = open('log.csv','a')
    f.write(str(sname)+','+ dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S") +'\n')
    f.close()
    return True

server.register_function(add_log, 'add_log')

def return_logs(op, dl, dr):
    dt_format = "%Y-%m-%d %H:%M:%S"
    dl = dt.datetime.strptime(dl, dt_format)
    dr = dt.datetime.strptime(dr, dt_format)
    log_files_list = [f"logs/{i}" for i in os.listdir(path="./logs")[1:]]
    df = pd.read_csv("log.csv", names=["operation", "dtime"])
    for i in log_files_list:
        df = pd.concat([df, pd.read_csv(i, names=["operation", "dtime"])])
    df["dtime"] = pd.to_datetime(df["dtime"].apply(lambda x : dt.datetime.strptime(x, "%Y-%m-%d %H:%M:%S")))
    x = [[i[0], str(i[1])] for i in df[(df.operation == op) & (df.dtime >= dl) & (df.dtime < dr)].values.tolist()]
    return x
server.register_function(return_logs, 'return_logs')


print ("Listening on port 8018...")
server.serve_forever()


Listening on port 8018...


127.0.0.1 - - [25/Sep/2023 10:26:06] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2023 10:26:10] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2023 10:26:14] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2023 10:26:18] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2023 10:26:22] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2023 10:26:27] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2023 10:26:31] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2023 10:26:35] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2023 10:26:39] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2023 10:26:43] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2023 10:26:47] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2023 10:26:53] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2023 10:26:58] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2023 10:27:02] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2023 10:27:06] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/202